In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload
import soma_extraction_utils as sm

Connecting celiib@10.28.0.34:3306


# debugging

In [2]:
import system_utils as su
new_submesh = su.decompress_pickle("./new_submesh")

In [3]:
new_submesh.split(only_watertight=False,repair=False)

array([<trimesh.Trimesh(vertices.shape=(83, 3), faces.shape=(178, 3))>],
      dtype=object)

In [4]:
import minfig
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons


In [5]:
minnie = minfig.configure_minnie(return_virtual_module=True)

# Old way of getting access to the virtual modules
# m65 = dj.create_virtual_module('minnie', 'microns_minnie65_02')

#New way of getting access to module
import datajoint as dj
from minfig import adapter_objects # included with wildcard imports
minnie = dj.create_virtual_module('minnie', 'microns_minnie65_02', add_objects=adapter_objects)

schema = dj.schema("microns_minnie65_02")
dj.config["enable_python_native_blobs"] = True
#(schema.jobs & "table_name='__baylor_segment_centroid_seg3'").delete()

INFO:datajoint.settings:Setting enable_python_native_blobs to True
INFO:datajoint.settings:Setting enable_python_native_blobs to True


# Fetching and Visualizing Meshes

In [6]:
# def get_decimated_mesh(seg_id,decimation_ratio=0.25):
#     key = dict(segment_id=seg_id,decimation_ratio=decimation_ratio)
#     new_mesh = (minnie.Decimation() & key).fetch1("mesh")
#     current_mesh_verts,current_mesh_faces = new_mesh.vertices,new_mesh.faces
#     return trimesh.Trimesh(vertices=current_mesh_verts,faces=current_mesh_faces)

# def get_seg_extracted_somas(seg_id):
#     key = dict(segment_id=seg_id)  
#     soma_vertices, soma_faces = (minnie.BaylorSegmentCentroid() & key).fetch("soma_vertices","soma_faces")
#     return [trimesh.Trimesh(vertices=v,faces=f) for v,f in zip(soma_vertices, soma_faces)]
# def get_soma_mesh_list(seg_id):
#     key = dict(segment_id=seg_id)  
#     soma_vertices, soma_faces,soma_run_time,soma_sdf = (minnie.BaylorSegmentCentroid() & key).fetch("soma_vertices","soma_faces","run_time","sdf")
#     s_meshes = [trimesh.Trimesh(vertices=v,faces=f) for v,f in zip(soma_vertices, soma_faces)]
#     s_times = list(soma_run_time)
#     s_sdfs = list(soma_sdf)
#     return [s_meshes,s_times,s_sdfs]

In [7]:
# multi_soma_seg_ids = np.unique(multi_soma_seg_ids)
# seg_id_idx = -2
# seg_id = multi_soma_seg_ids[seg_id_idx]

# dec_mesh = get_decimated_mesh(seg_id)
# curr_soma_meshes = get_seg_extracted_somas(seg_id)
# curr_soma_mesh_list = get_soma_mesh_list(seg_id)

# import skeleton_utils as sk
# sk.graph_skeleton_and_mesh(main_mesh_verts=dec_mesh.vertices,
#                            main_mesh_faces=dec_mesh.faces,
#                         other_meshes=curr_soma_meshes,
#                           other_meshes_colors="red")

# Exploring the New Query and Seeing if matches up with nucleus id:

In [45]:
decimation_ratio = 0.25

valid_segment_ids_with_nucleus_id = dj.U("segment_id") & (minnie.NucleusID() & "segment_id>0")
segments = (minnie.Mesh() & f'n_vertices  > {np.str(np.round(np.exp(12)).astype(np.int))}' & f'n_vertices  < {np.str(np.round(np.exp(15)).astype(np.int))}')
key_source =  minnie.Decimation.proj(decimation_version='version') & segments.proj() & f"decimation_ratio={decimation_version}" & valid_segment_ids_with_nucleus_id
key_source

segment_id segment id unique within each Segmentation,decimation_version,decimation_ratio ratio of remaining mesh vertices/faces (which ones depends on what metric the decimation technique uses)
864691135635239593,0,0.25
864691135618175635,0,0.25
864691135584809548,0,0.25
864691135679304325,0,0.25
864691136143741847,0,0.25
864691136148842647,0,0.25
864691136143284631,0,0.25
864691136745551332,0,0.25
864691136315868311,0,0.25
864691136324529047,0,0.25


In [46]:
valid_segment_ids_with_nucleus = key_source.fetch("segment_id")
valid_segment_ids_with_nucleus

array([864691135635239593, 864691135618175635, 864691135584809548, ...,
       864691135818851060, 864691135984827392, 864691135252751535])

In [ ]:
true_negatives=[4,6]
false_positives=[13,15]

In [254]:
du = reload(du)
seg_id_idx = 21
seg_id = valid_segment_ids_with_nucleus[seg_id_idx]
du.plot_decimated_mesh_with_somas(seg_id)

#error_poisson_somas = du.get_seg_extracted_somas(seg_id)
error_mesh = du.get_decimated_mesh(seg_id)

# error_backtrack_somas = sm.original_mesh_soma(
#                     mesh = error_mesh,
#                     soma_meshes=error_poisson_somas,
#                     sig_th_initial_split=15)

import trimesh_utils as tu
sm = reload(sm)
tu = reload(tu)
meshlab= reload(meshlab)

soma_data = sm.extract_soma_center(seg_id,
                      error_mesh.vertices,
                      error_mesh.faces,
                                   side_length_ratio_threshold=6,
                                  backtrack_soma_mesh_to_original=True,
                                   boundary_vertices_threshold=None,
                        poisson_backtrack_distance_threshold=None,
                                   close_holes=False
                                  )


INFO:datajoint.settings:Setting enable_python_native_blobs to True
/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


Current Arguments Using (adjusted for decimation):
 large_mesh_threshold= 15000.0 
large_mesh_threshold_inner = 10000.0 
soma_size_threshold = 1250.0 
soma_size_threshold_max = 12000.0
outer_decimation_ratio = 0.25
inner_decimation_ratio = 0.25
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 7382
xvfb-run -n 7382 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoint/Soma_Extraction/864691135783403116/neuron_864691135783403116.off -o /notebooks/Platinum_Datajoint/Soma_Extraction/864691135783403116/neuron_864691135783403116_decimated.off -s /notebooks/Platinum_Datajoint/Soma_Extraction/864691135783403116/decimation_meshlab_25147971.mls
Total found significant pieces before Poisson = [<trimesh.Trimesh(vertices.shape=(16864, 3), faces.shape=(33739, 3))>]
----- working on large mesh #0: <trimesh.Trimesh(vertices.shape=(16864, 3), faces.shape=(33739, 3))>
pre_largest_mesh_path = /notebooks/Platinum_Datajoint/Soma_Extraction/864691135783403116/ne

Total found significant pieces AFTER Poisson = [<trimesh.Trimesh(vertices.shape=(35254, 3), faces.shape=(70524, 3))>]
----- working on mesh after poisson #0: <trimesh.Trimesh(vertices.shape=(35254, 3), faces.shape=(70524, 3))>
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 9969
xvfb-run -n 9969 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoint/Soma_Extraction/864691135783403116/neuron_864691135783403116_decimated_largest_piece_poisson_largest_inner.off -o /notebooks/Platinum_Datajoint/Soma_Extraction/864691135783403116/neuron_864691135783403116_decimated_largest_piece_poisson_largest_inner_decimated.off -s /notebooks/Platinum_Datajoint/Soma_Extraction/864691135783403116/decimation_meshlab_25406536.mls
done exporting decimated mesh: neuron_864691135783403116_decimated_largest_piece_poisson_largest_inner.off
1) Starting: Mesh importing and Pymesh fix
loading mesh from vertices and triangles array
1) Finished: Mesh importing and Pymesh f

/meshAfterParty/trimesh_utils.py:299: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  ordered_components = np.array([components[i] for i in ordered_indices])


LEAVING LOOP, MESH VALIDATED
Using port = 9023
xvfb-run -n 9023 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoint/Soma_Extraction/Poisson_temp/neuron_209443.off -o /notebooks/Platinum_Datajoint/Soma_Extraction/Poisson_temp/neuron_209443_poisson.off -s /notebooks/Platinum_Datajoint/Soma_Extraction/Poisson_temp/poisson_969690.mls


removed temporary input file: /notebooks/Platinum_Datajoint/Soma_Extraction/Poisson_temp/neuron_209443.off
removed temporary output file: /notebooks/Platinum_Datajoint/Soma_Extraction/Poisson_temp/neuron_209443_poisson.off
/notebooks/Platinum_Datajoint/Soma_Extraction/Poisson_temp/poisson_969690.mls is being deleted....
Inside sphere validater: ratio_val = 2.436553946144553


In [255]:
print(f"seg_id_idx = {seg_id_idx}")
soma_meshes,_,_ = soma_data
print(f"len(soma_meshes) = {len(soma_meshes)}")

import skeleton_utils as sk
sk.graph_skeleton_and_mesh(other_meshes=[error_mesh] + soma_meshes,
                          other_meshes_colors=["yellow","red"],
                          mesh_alpha=1
                        )

seg_id_idx = 21
len(soma_meshes) = 1


/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [256]:
print(f"seg_id_idx = {seg_id_idx}")
import skeleton_utils as sk
sk.graph_skeleton_and_mesh(other_meshes=soma_meshes,
                          other_meshes_colors=["red"],
                          mesh_alpha=1
                        )

seg_id_idx = 21


In [ ]:
# Remove Faces From Non Manifold Edges

In [186]:
# import meshlab
# #meshlab = reload(meshlab)
# fill_hole_obj = meshlab.FillHoles(max_hole_size=2000,
#                  self_itersect_faces=False)

# error_mesh_closed_holes,_ = fill_hole_obj(vertices=error_mesh.vertices,
#              faces=error_mesh.faces)

# sk.graph_skeleton_and_mesh(other_meshes=[error_mesh,error_mesh_closed_holes],
#                           other_meshes_colors=["black","red"],
#                           mesh_alpha=1)

self_itersect_faces = False
current_script = {'Remove Duplicate Vertices': {}, 'Remove Faces from Non Manifold Edges': {}, 'Close Holes': {'MaxHoleSize': {'type': 'RichInt', 'value': '2000'}, 'Selected': {'type': 'RichBool', 'value': 'false'}, 'NewFaceSelected': {'type': 'RichBool', 'value': 'false'}, 'SelfIntersection': {'type': 'RichBool', 'value': 'false'}}}
IN INPUT FILE VALIDATION LOOP
LEAVING LOOP, MESH VALIDATED
Using port = 2285
xvfb-run -n 2285 -s "-screen 0 800x600x24" meshlabserver $@  -i /notebooks/Platinum_Datajoint/Soma_Extraction/temp/neuron_None.off -o /notebooks/Platinum_Datajoint/Soma_Extraction/temp/neuron_None_fill_holes.off -s /notebooks/Platinum_Datajoint/Soma_Extraction/temp/fill_holes_627033.mls
removed temporary input file: /notebooks/Platinum_Datajoint/Soma_Extraction/temp/neuron_None.off
removed temporary output file: /notebooks/Platinum_Datajoint/Soma_Extraction/temp/neuron_None_fill_holes.off


In [180]:
str(False).lower()

'false'

# Plots the soma

In [49]:
soma_meshes

[]

/usr/local/lib/python3.6/dist-packages/traittypes/traittypes.py:101: UserWarning: Given trait value dtype "float64" does not match required type "float64". A coerced copy has been created.
  np.dtype(self.dtype).name))


In [44]:
sk.graph_skeleton_and_mesh(other_meshes=soma_meshes)

Exception: No meshes or skeletons passed to the plotting funciton

# Debugging the new soma mesh finder 

# -- getting neurons to help test on -- 

In [13]:
# sm = reload(sm)

In [14]:
# import neuron_utils as nru
# filepath = "/notebooks/test_neurons/meshafterparty_processed/12345_double_soma_meshafterparty_fixed_connectors_and_spines"
# double_neuron = nru.decompress_neuron(
#     filepath=filepath,
#     original_mesh=filepath,
# )



In [15]:
# sm = reload(sm)
# double_neuron_poisson_somas = sm.extract_soma_center(double_neuron.segment_id,
#                                                     double_neuron.mesh.vertices,
#                                                     double_neuron.mesh.faces)
# d_neuron_poisson_somas,_ , _ = double_neuron_poisson_somas
# d_neuron_backtrack_somas,_,_ = double_neuron.get_somas()

# -- Getting other meshes to try -- 

In [16]:
# exc_1 = tu.load_mesh_no_processing("/notebooks/test_neurons/spine_detection/95442489112204099_excitatory_7.off")
# soma_data = sm.extract_soma_center(1234,exc_1.vertices,exc_1.faces)

In [17]:
# exc_1_poisson_somas,_,_ = soma_data
# exc_1_backtrack_somas = sm.original_mesh_soma(
#                     mesh = exc_1,
#                     soma_meshes=exc_1_poisson_somas,
#                     sig_th_initial_split=15)

In [18]:
# exc_1_poisson_somas,exc_1_backtrack_somas

In [19]:
# sk.graph_skeleton_and_mesh(other_meshes=exc_1_poisson_somas+exc_1_backtrack_somas,
#                           other_meshes_colors=["black","red"])

# --- Checking Neurons that should be processed -- 

In [20]:
# valid_segment_ids_with_nucleus_id = dj.U("segment_id") & (minnie.NucleusID() & "segment_id>0")

In [21]:
# len(minnie.NucleusID() & all_nucleus_id_segs)

In [22]:
# len(minnie.NucleusID() & "segment_id>0")

In [23]:
# minnie.NucleusCorePostsyn() & "n_soma = 2"# robust table 

# Apply a check that looks and sees if has a border that is too big